# Coleta e preparação de dados

In [1]:
!pip install pandas==2.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.0 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 2.0.0 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 2.0.0 which is incompatible.
xarray 2025.1.2 requires pandas>=2.1, but you have pandas 2.0.0 which is incompatible.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://github.com/lukaswyllis/formacao-avancando-em-data-science-com-python/raw/refs/heads/master/dados-curso4/relatorio_vendas.csv')

In [4]:
df.head()

,data_pedido,modo_envio,nome_cliente,segmento_cliente,cidade,estado,regiao,departamento,tipo_produto,vendas,quantidade,lucro
0,2018-11-09,Econômica,Thiago Silveira,B2C,Ribeirão Preto,São Paulo,Sudeste,Materiais de construção,encanamentos,890.66,2,142.51
1,2018-11-09,Econômica,Thiago Silveira,B2C,Ribeirão Preto,São Paulo,Sudeste,Materiais de construção,ferramentas,2488.60,3,746.58
2,2018-06-13,Econômica,Giovanna Lima,B2B,Rio de Janeiro,Rio de Janeiro,Sudeste,Jardinagem e paisagismo,sementes,49.71,2,23.36
3,2017-10-12,Entrega padrão,Ana Júlia da Cruz,B2C,Foz do Iguaçu,Paraná,Sul,Materiais de construção,materiais de revestimento,3255.76,5,-1302.31
4,2017-10-12,Entrega padrão,Ana Júlia da Cruz,B2C,Foz do Iguaçu,Paraná,Sul,Jardinagem e paisagismo,vasos,76.05,2,8.56


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8940 entries, 0 to 8939
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   data_pedido       8940 non-null   object 
 1   modo_envio        8940 non-null   object 
 2   nome_cliente      8940 non-null   object 
 3   segmento_cliente  8940 non-null   object 
 4   cidade            8940 non-null   object 
 5   estado            8940 non-null   object 
 6   regiao            8940 non-null   object 
 7   departamento      8940 non-null   object 
 8   tipo_produto      8940 non-null   object 
 9   vendas            8940 non-null   float64
 10  quantidade        8940 non-null   int64  
 11  lucro             8940 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 838.3+ KB


In [34]:
df['data_pedido'] = pd.to_datetime(df['data_pedido'])

# Missão 1: Time de vendas

Precisamos construir uma visualização para indicar os **10 maiores clientes por total de venda**.

Com isso, a equipe de vendas pode pensar em estratégias para manter esses clientes comprando em nossa loja.

In [6]:
df_cliente_vendas = df.groupby(['nome_cliente'])['vendas'].sum().nlargest(10).copy()
df_cliente_vendas = df_cliente_vendas.reset_index()
df_cliente_vendas.columns = ['Clientes', 'Vendas']
df_cliente_vendas['Ranking'] = df_cliente_vendas.index + 1
df_cliente_vendas.set_index(df_cliente_vendas['Ranking'], inplace=True)
df_cliente_vendas.drop('Ranking', axis=1, inplace=True)
df_cliente_vendas

,Clientes,Vendas
Ranking,,
1,Maria Luiza Almeida,64777.54
2,Ana Julia Pinto,51398.95
3,Ryan Farias,48178.94
4,Heitor da Mata,47610.85
5,Maria Clara Gonçalves,46946.42
6,Raquel Freitas,44826.26
7,Davi Ramos,43769.21
8,Amanda Melo,42354.27
9,Alexia Ribeiro,41238.84


In [7]:
s = df_cliente_vendas.style
s.format({'Vendas': 'R$ {:,.2f}'})
s

,Clientes,Vendas
Ranking,,
1,Maria Luiza Almeida,"R$ 64,777.54"
2,Ana Julia Pinto,"R$ 51,398.95"
3,Ryan Farias,"R$ 48,178.94"
4,Heitor da Mata,"R$ 47,610.85"
5,Maria Clara Gonçalves,"R$ 46,946.42"
6,Raquel Freitas,"R$ 44,826.26"
7,Davi Ramos,"R$ 43,769.21"
8,Amanda Melo,"R$ 42,354.27"
9,Alexia Ribeiro,"R$ 41,238.84"


## Desafio 1
O primeiro desafio é construir uma visualização que permita identificar quais países mais solicitaram produtos por pedidos para que assim possa ser feito um estudo quanto a distribuição e logística de produtos.

In [8]:
df_loja_livro_filmes = pd.read_csv('https://raw.githubusercontent.com/lukaswyllis/formacao-avancando-em-data-science-com-python/refs/heads/master/dados-curso4/loja_livro_filmes.csv')

In [9]:
df_paises = df_loja_livro_filmes.groupby('pais')['unidades'].sum().nlargest(10).copy()
df_paises = df_paises.reset_index()
df_paises.columns = ['País', 'Unidades']
df_paises['Rank'] = df_paises.index + 1
df_paises.set_index('Rank', inplace=True)
s_paises = df_paises.style
s_paises

,País,Unidades
Rank,,
1,Argentina,5591
2,Equador,5566
3,Paraguai,5564
4,Peru,5550
5,Venezuela,5529
6,Colômbia,5504
7,Brasil,5473
8,Uruguai,5377
9,Chile,5304


# Missão 2: Time comercial

Precisamos construir uma visualização que permita associar o **total ganho em vendas e o total lucrado por tipo de produto**.

Isso vai permitir que o time comercial possa focar em estratégias para alavancar as rendas que ganham com cada tipo de produto vendido na loja.

In [10]:
df_venda_lucro = df.groupby(['tipo_produto'])[['vendas', 'lucro']].sum()
df_venda_lucro.index.name = 'Tipo Produto'
df_venda_lucro

,vendas,lucro
Tipo Produto,,
decoração de jardim,82680.87,19880.86
encanamentos,373224.39,-11243.39
equipamentos de limpeza,542304.55,17448.30
ferramentas,995159.43,82042.91
ferramentas automotivas,502109.33,126660.54
ferramentas de jardinagem,648880.47,106408.80
fertilizantes,53144.55,22509.86
iluminação,275229.82,40531.09
materiais de paisagismo,150552.80,-3823.67


In [11]:
estilo_produto = df_venda_lucro.style
estilo_produto

,vendas,lucro
Tipo Produto,,
decoração de jardim,82680.870000,19880.860000
encanamentos,373224.390000,-11243.390000
equipamentos de limpeza,542304.550000,17448.300000
ferramentas,995159.430000,82042.910000
ferramentas automotivas,502109.330000,126660.540000
ferramentas de jardinagem,648880.470000,106408.800000
fertilizantes,53144.550000,22509.860000
iluminação,275229.820000,40531.090000
materiais de paisagismo,150552.800000,-3823.670000


In [12]:
estilo_produto.format('R$ {:,.2f}')\
              .highlight_max(color='darkgreen')\
              .highlight_min(color='#F16165')

,vendas,lucro
Tipo Produto,,
decoração de jardim,"R$ 82,680.87","R$ 19,880.86"
encanamentos,"R$ 373,224.39","R$ -11,243.39"
equipamentos de limpeza,"R$ 542,304.55","R$ 17,448.30"
ferramentas,"R$ 995,159.43","R$ 82,042.91"
ferramentas automotivas,"R$ 502,109.33","R$ 126,660.54"
ferramentas de jardinagem,"R$ 648,880.47","R$ 106,408.80"
fertilizantes,"R$ 53,144.55","R$ 22,509.86"
iluminação,"R$ 275,229.82","R$ 40,531.09"
materiais de paisagismo,"R$ 150,552.80","R$ -3,823.67"


In [13]:
estilo_produto.format('R$ {:,.2f}').background_gradient(cmap='Greens')
estilo_produto

,vendas,lucro
Tipo Produto,,
decoração de jardim,"R$ 82,680.87","R$ 19,880.86"
encanamentos,"R$ 373,224.39","R$ -11,243.39"
equipamentos de limpeza,"R$ 542,304.55","R$ 17,448.30"
ferramentas,"R$ 995,159.43","R$ 82,042.91"
ferramentas automotivas,"R$ 502,109.33","R$ 126,660.54"
ferramentas de jardinagem,"R$ 648,880.47","R$ 106,408.80"
fertilizantes,"R$ 53,144.55","R$ 22,509.86"
iluminação,"R$ 275,229.82","R$ 40,531.09"
materiais de paisagismo,"R$ 150,552.80","R$ -3,823.67"


In [14]:
cabecalho = {
    'selector':'th',
    'props':'font-weigth: bold; font-family: Arial; text-align: center; text-transform: capitalize;'
}

In [15]:
estilo_produto.set_table_styles([cabecalho], overwrite=False)

,vendas,lucro
Tipo Produto,,
decoração de jardim,"R$ 82,680.87","R$ 19,880.86"
encanamentos,"R$ 373,224.39","R$ -11,243.39"
equipamentos de limpeza,"R$ 542,304.55","R$ 17,448.30"
ferramentas,"R$ 995,159.43","R$ 82,042.91"
ferramentas automotivas,"R$ 502,109.33","R$ 126,660.54"
ferramentas de jardinagem,"R$ 648,880.47","R$ 106,408.80"
fertilizantes,"R$ 53,144.55","R$ 22,509.86"
iluminação,"R$ 275,229.82","R$ 40,531.09"
materiais de paisagismo,"R$ 150,552.80","R$ -3,823.67"


## **Desafio 2: hora da prática**

Aprendemos a construir uma visualização capaz de destacar elementos através de algumas funções built-in. E, nesse desafio, podemos aplicar algumas dessas funções.

O objetivo desse desafio é construir uma visualização que permita **identificar quais categorias de produto geraram mais e menos custos para a loja**.

In [16]:
df_categoria = df_loja_livro_filmes.groupby(['categoria'])[['custo_produto']].sum()
estilo_categoria = df_categoria.style
estilo_categoria.format('R$ {:,.2f}')\
              .highlight_max(color='darkgreen')\
              .highlight_min(color='#F16165')

,custo_produto
categoria,
BlueRay,"R$ 14,191.36"
CD,"R$ 10,514.28"
Coleção,"R$ 73,128.59"
Livro,"R$ 78,213.80"


# Missão 3: Time de logística

Precisamos construir uma visualização que permita entender a **distribuição de pedidos por região do Brasil**.

Isso vai permitir que o time logística entenda onde é mais interessante focar material e pessoas.

In [17]:
df_regiao = pd.DataFrame(df['regiao'].value_counts())
df_regiao.columns = ['Nº pedidos']
df_regiao.index.name = 'Região'
df_regiao

,Nº pedidos
Região,
Sudeste,4470
Nordeste,2075
Centro-Oeste,983
Norte,779
Sul,633


In [18]:
import numpy as np

In [19]:
porcentagem = np.array(df_regiao['Nº pedidos'].to_numpy())
porcentagem = porcentagem/porcentagem.sum() * 100
porcentagem

array([50.        , 23.21029083, 10.99552573,  8.71364653,  7.08053691])

In [20]:
df_regiao['Porcentagem'] = porcentagem

In [21]:
df_regiao

,Nº pedidos,Porcentagem
Região,,
Sudeste,4470,50.000000
Nordeste,2075,23.210291
Centro-Oeste,983,10.995526
Norte,779,8.713647
Sul,633,7.080537


In [22]:
estilo_regiao = df_regiao.style

In [23]:
estilo_regiao

,Nº pedidos,Porcentagem
Região,,
Sudeste,4470,50.000000
Nordeste,2075,23.210291
Centro-Oeste,983,10.995526
Norte,779,8.713647
Sul,633,7.080537


In [24]:
cabecalho = {
    'selector': 'th',
    'props': 'font-family: Helvetica; color: #dddd55; background-color: #003641; text-align: center;'
}

celulas = {
    'selector': 'td',
    'props': 'font-family: Helvetica; color: white; background-color: #003641; text-align: left;'
}

In [25]:
estilo_regiao.set_table_styles([cabecalho, celulas])

,Nº pedidos,Porcentagem
Região,,
Sudeste,4470,50.000000
Nordeste,2075,23.210291
Centro-Oeste,983,10.995526
Norte,779,8.713647
Sul,633,7.080537


In [26]:
estilo_regiao.format({'Porcentagem': '{:,.2f}%'})\
             .bar(subset='Porcentagem', vmin=0, vmax=100.0, color='#00AE9D')

,Nº pedidos,Porcentagem
Região,,
Sudeste,4470,50.00%
Nordeste,2075,23.21%
Centro-Oeste,983,11.00%
Norte,779,8.71%
Sul,633,7.08%


## Desafio 3: hora da prática
Aprendemos a adicionar uma visualização gráfica na nossa tabela através da função `bar`. Nesse desafio, podemos aplicar essa função para treinarmos a construção da visualização com adição gráfica de uma forma bem similar ao que fizemos na aula.

O desafio é construir uma visualização que permita **informar a quantidade e distribuição de pedidos por tipo de desconto**. Isso irá permitir que os setores da empresa entendam a demanda de produtos em cada ação promocional.*texto em itálico*

In [27]:
df_desconto = pd.DataFrame(df_loja_livro_filmes['tipo_desconto'].value_counts())
df_desconto.columns = ['Quantidade']
df_desconto.index.name = 'Tipo de Desconto'
df_desconto['Porcentagem'] = df_desconto['Quantidade'] / df_desconto['Quantidade'].sum() * 100
df_desconto

,Quantidade,Porcentagem
Tipo de Desconto,,
Sem desconto,13347,53.623945
Cyber Monday,6922,27.810366
Black Friday,1986,7.979108
Liquidação,1487,5.974287
Data especial,1148,4.612294


In [28]:
s_desconto = df_desconto.style
cabecalho = {
    'selector': 'th',
    'props': 'font-family: Helvetica; color: orange; background-color: purple; text-align: center;'
}

celulas = {
    'selector': 'td',
    'props': 'font-family: Helvetica; color: orange; background-color: purple; text-align: left;'
}
s_desconto.set_table_styles([cabecalho, celulas])
s_desconto.format({'Porcentagem': '{:,.2f}%'})\
             .bar(subset='Porcentagem', vmin=0, vmax=100.0, color='black')

,Quantidade,Porcentagem
Tipo de Desconto,,
Sem desconto,13347,53.62%
Cyber Monday,6922,27.81%
Black Friday,1986,7.98%
Liquidação,1487,5.97%
Data especial,1148,4.61%


# Missão 4: Time de logística

Novamente o time de logística deseja repensar sua estrutura e organização de fornecimento de produtos e para isso, foi pensada em uma visualização que permita identificar padrão de **quantidade de produtos solicitados por mês por departamento**.

In [39]:
df_meses = df.copy()
df_meses = df_meses.sort_values('data_pedido')
df_meses['meses'] = df_meses['data_pedido'].dt.strftime('%Y - %b')
df_meses = df_meses.reset_index()

In [44]:
df_meses

,index,data_pedido,modo_envio,nome_cliente,segmento_cliente,cidade,estado,regiao,departamento,tipo_produto,vendas,quantidade,lucro,meses
0,7171,2016-01-04,Entrega padrão,Ana Júlia Monteiro,B2C,Belo Horizonte,Minas Gerais,Sudeste,Jardinagem e paisagismo,pesticidas,55.92,2,18.87,2016 - Jan
1,651,2016-01-05,Entrega padrão,Maria Cecília Jesus,B2B,Botucatu,São Paulo,Sudeste,Jardinagem e paisagismo,ferramentas de jardinagem,12.04,2,-18.66,2016 - Jan
2,650,2016-01-05,Entrega padrão,Maria Cecília Jesus,B2B,Botucatu,São Paulo,Sudeste,Jardinagem e paisagismo,vasos,927.30,3,-220.23,2016 - Jan
3,649,2016-01-05,Entrega padrão,Maria Cecília Jesus,B2B,Botucatu,São Paulo,Sudeste,Jardinagem e paisagismo,sementes,40.07,3,14.52,2016 - Jan
4,1576,2016-01-06,Entrega padrão,Marcelo Rezende,B2C,Brasília,Distrito Federal,Centro-Oeste,Jardinagem e paisagismo,decoração de jardim,66.42,3,16.61,2016 - Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8935,796,2019-12-31,Entrega padrão,Lara Pinto,B2C,São Paulo,São Paulo,Sudeste,Jardinagem e paisagismo,ferramentas de jardinagem,179.44,3,67.29,2019 - Dec
8936,795,2019-12-31,Entrega padrão,Lara Pinto,B2C,São Paulo,São Paulo,Sudeste,Automotivo,pneus,309.16,7,9.27,2019 - Dec
8937,794,2019-12-31,Entrega padrão,Lara Pinto,B2C,São Paulo,São Paulo,Sudeste,Materiais de construção,encanamentos,1098.66,4,41.20,2019 - Dec
8938,1151,2019-12-31,Entrega padrão,Cauê Martins,B2B,Osasco,São Paulo,Sudeste,Jardinagem e paisagismo,ferramentas de jardinagem,47.27,2,15.36,2019 - Dec


In [45]:
df_vendas_mensais = df_meses.pivot_table(index='departamento', columns='meses', values='quantidade', aggfunc='sum', sort=False)

In [49]:
estilo_mensal = df_vendas_mensais.style
estilo_mensal.set_sticky(axis='index')

meses,2016 - Jan,2016 - Feb,2016 - Mar,2016 - Apr,2016 - May,2016 - Jun,2016 - Jul,2016 - Aug,2016 - Sep,2016 - Oct,2016 - Nov,2016 - Dec,2017 - Jan,2017 - Feb,2017 - Mar,2017 - Apr,2017 - May,2017 - Jun,2017 - Jul,2017 - Aug,2017 - Sep,2017 - Oct,2017 - Nov,2017 - Dec,2018 - Jan,2018 - Feb,2018 - Mar,2018 - Apr,2018 - May,2018 - Jun,2018 - Jul,2018 - Aug,2018 - Sep,2018 - Oct,2018 - Nov,2018 - Dec,2019 - Jan,2019 - Feb,2019 - Mar,2019 - Apr,2019 - May,2019 - Jun,2019 - Jul,2019 - Aug,2019 - Sep,2019 - Oct,2019 - Nov,2019 - Dec
departamento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Jardinagem e paisagismo,161,87,277,323,270,273,348,339,549,276,716,525,97,131,278,271,269,321,268,322,663,260,746,628,190,177,260,338,463,473,413,380,757,402,727,770,361,205,458,415,542,490,423,561,929,617,946,989
Automotivo,45,20,46,107,81,83,91,95,154,85,236,154,55,44,74,111,94,59,88,117,180,100,179,235,59,49,112,72,124,118,117,123,181,103,253,189,109,63,167,87,150,170,168,136,280,205,331,252
Materiais de construção,60,23,102,69,91,114,104,101,182,94,213,269,93,38,100,85,76,120,128,106,194,129,259,258,91,67,127,128,171,117,132,113,227,171,308,294,83,67,107,131,172,151,162,132,282,213,312,395


In [50]:
colunas = {
    'selector': '.col_heading',
    'props': 'font-weight: normal; font-family: Arial'
}

tabelas = {
    'selector': 'td, th',
    'props': 'text-align: left'
}

index = {
    'selector': '.index_name',
    'props': 'font-weight: bold; font-family: Arial; text-align: right'
}

estilo_mensal.set_table_styles([colunas, tabelas, index], overwrite=False)

meses,2016 - Jan,2016 - Feb,2016 - Mar,2016 - Apr,2016 - May,2016 - Jun,2016 - Jul,2016 - Aug,2016 - Sep,2016 - Oct,2016 - Nov,2016 - Dec,2017 - Jan,2017 - Feb,2017 - Mar,2017 - Apr,2017 - May,2017 - Jun,2017 - Jul,2017 - Aug,2017 - Sep,2017 - Oct,2017 - Nov,2017 - Dec,2018 - Jan,2018 - Feb,2018 - Mar,2018 - Apr,2018 - May,2018 - Jun,2018 - Jul,2018 - Aug,2018 - Sep,2018 - Oct,2018 - Nov,2018 - Dec,2019 - Jan,2019 - Feb,2019 - Mar,2019 - Apr,2019 - May,2019 - Jun,2019 - Jul,2019 - Aug,2019 - Sep,2019 - Oct,2019 - Nov,2019 - Dec
departamento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Jardinagem e paisagismo,161,87,277,323,270,273,348,339,549,276,716,525,97,131,278,271,269,321,268,322,663,260,746,628,190,177,260,338,463,473,413,380,757,402,727,770,361,205,458,415,542,490,423,561,929,617,946,989
Automotivo,45,20,46,107,81,83,91,95,154,85,236,154,55,44,74,111,94,59,88,117,180,100,179,235,59,49,112,72,124,118,117,123,181,103,253,189,109,63,167,87,150,170,168,136,280,205,331,252
Materiais de construção,60,23,102,69,91,114,104,101,182,94,213,269,93,38,100,85,76,120,128,106,194,129,259,258,91,67,127,128,171,117,132,113,227,171,308,294,83,67,107,131,172,151,162,132,282,213,312,395


# Missão 5: Relatório de performance

O relatório de performance é um documento muito importante para a empresa. Nele, todos que trabalham na loja podem entender como anda a empresa. Por conta disso, tabelas bem construídas são essenciais para mostrar os dados da melhor forma.

Portanto, precisamos construir uma visualização que permita entender a **relação entre tipos de clientes e modo de envio de produtos de acordo com as vendas**. Com isso, será possível destacar no relatório qual o modo de envio e quais tipos de modo de envio mais geraram ganho em vendas.

In [52]:
df_cliente = df.pivot_table(index='segmento_cliente', columns='modo_envio', values='vendas', aggfunc='sum')
df_cliente

modo_envio,24 horas,Econômica,Entrega padrão,Envio rápido
segmento_cliente,,,,
B2B,217466.05,718462.52,1884740.78,602049.56
B2C,182531.80,729176.13,2248286.96,498295.55


In [54]:
df_cliente['Total'] = df_cliente.sum(axis=1)
df_cliente.loc['Total'] = df_cliente.sum(axis=0)
df_cliente

modo_envio,24 horas,Econômica,Entrega padrão,Envio rápido,Total
segmento_cliente,,,,,
B2B,217466.05,718462.52,1884740.78,602049.56,6845437.82
B2C,182531.80,729176.13,2248286.96,498295.55,7316580.88
Total,399997.85,1447638.65,4133027.74,1100345.11,14162018.70


In [55]:
compra_cliente = df_cliente.style.format('{:,.2f}')
compra_cliente

modo_envio,24 horas,Econômica,Entrega padrão,Envio rápido,Total
segmento_cliente,,,,,
B2B,"217,466.05","718,462.52","1,884,740.78","602,049.56","6,845,437.82"
B2C,"182,531.80","729,176.13","2,248,286.96","498,295.55","7,316,580.88"
Total,"399,997.85","1,447,638.65","4,133,027.74","1,100,345.11","14,162,018.70"


In [61]:
tabela = {
    'selector': 'td,th:not(.index_name)',
    'props': 'font-weight: normal; font-family: Arial; text-align: center; background-color: black'
}
index = {
    'selector': '.index_name',
    'props': 'font-weight: normal; text-align: right; font-style: italic; color: white'
}
compra_cliente.set_table_styles([tabela,index])

modo_envio,24 horas,Econômica,Entrega padrão,Envio rápido,Total
segmento_cliente,,,,,
B2B,"217,466.05","718,462.52","1,884,740.78","602,049.56","6,845,437.82"
B2C,"182,531.80","729,176.13","2,248,286.96","498,295.55","7,316,580.88"
Total,"399,997.85","1,447,638.65","4,133,027.74","1,100,345.11","14,162,018.70"


In [62]:
compra_cliente.set_table_styles({
    'Total':[{
        'selector': 'th',
        'props': 'border-top: 1px solid #D3D3D3'
    },
    {
        'selector': 'td',
        'props': 'border-top: 1px solid #D3D3D3'
    }],
    'B2B': [{
        'selector': 'th',
        'props': 'border-top: 1px solid #D3D3D3'
    },
    {
        'selector': 'td',
        'props': 'border-top: 1px solid #D3D3D3'
    }]
}, overwrite=False, axis=1)

modo_envio,24 horas,Econômica,Entrega padrão,Envio rápido,Total
segmento_cliente,,,,,
B2B,"217,466.05","718,462.52","1,884,740.78","602,049.56","6,845,437.82"
B2C,"182,531.80","729,176.13","2,248,286.96","498,295.55","7,316,580.88"
Total,"399,997.85","1,447,638.65","4,133,027.74","1,100,345.11","14,162,018.70"


In [66]:
compra_cliente.set_table_styles({
    'Total': [{
        'selector': '.true',
        'props': 'background-color: #2E2E2E'
    }]
}, overwrite=False, axis=0)

cores_coluna = pd.DataFrame(['false', 'true', 'false'], index=df_cliente['Total'].index, columns=['Total'])

compra_cliente.set_td_classes(cores_coluna)

modo_envio,24 horas,Econômica,Entrega padrão,Envio rápido,Total
segmento_cliente,,,,,
B2B,"217,466.05","718,462.52","1,884,740.78","602,049.56","6,845,437.82"
B2C,"182,531.80","729,176.13","2,248,286.96","498,295.55","7,316,580.88"
Total,"399,997.85","1,447,638.65","4,133,027.74","1,100,345.11","14,162,018.70"
